In [17]:
from IPython.display import HTML

# 18 gecs: Improving Rigid Body Physics Simulations using LCPs

## Problem Description

Rigid body physics simulations, which approximate real-world objects as nondeformable distributions of mass, are frequently used in video games and other computer animations to produce reasonable-looking approximations of actual physics. A variety of techniques exist for modeling the interactions of rigid bodies; this project deals with the implementation of a collision management system called "speculative contacts." We chose this system because it has the following desirable properties:

1. It is a "continuous" collision management system; i.e., no matter how fast objects are moving, they won't teleport through other objects.
2. It supports "stable stacks;" i.e., multiple objects can be in a constantly colliding state without having a "jittery" appearance.
3. Unlike most collision management systems with properties 1. and 2., it is simple to describe and implement.

It can be described as follows:
* At the beginning of each timestep, the simulation is broken down into a set of “contacts.” A contact represents two points that might collide with one another. If the two points’ movements during this timestep (as determined by their objects’ velocities) overlap when projected onto the line connecting the two points (the “normal” line), the contact is in a “colliding” state.
* Colliding contacts are "fixed" in a physically reasonable manner by applying some impulse $i$ to both points along their normal line such that both points "repel" one another, and do so *just* enough that their movements during this timestep no longer overlap when projected onto the normal line. Excess repulsion is called "overcorrection."
* Before the simulation is advanced to the next timestep, all collisions must be fixed without any overcorrection.

## Our Approach

We compared the following implementations of speculative contacts:
* `greedy` - At the beginning of every timestep, look at every contact sequentially. When you find a colliding contact, apply an repelling impulse $i$ to both bodies such that the contact is no longer colliding, and has no overcorrection.
* `greedyWithCorrections(k)` - At the beginning of every timestep, run `greedy`. Then, look through every contact again, fixing collisions a la `greedy`, but also detecting and fixing overcorrection (if a contact has had some $i$ applied to it and the distance between its points at the start of the next timestep will be greater than zero, it is overcorrected; this is fixed by applying an attractive impulse $i'$ to both bodies such that the total impulse applied is still nonattractive, but the points' distance at the start of the next timestep will be as close to zero as possible). This iterate-and-fix-overcorrection strategy is repeated `k` times - the idea is that as `k` increases, the result gets less and less wrong.
* `LCP` - Determine the $i$ to apply to each contact by solving the following LCP:

$$
\textrm{minimize } (1/2)x^TPx + q^Tx
$$
$$
\textrm{subject to } Gx \leq h
$$
* $x$ contains the $i$ to apply to each contact; $x_j > 0 \implies$ a repelling impulse; $x_j < 0 \implies$ an attractive impulse
* $G$ is of dimension $2|x|$ by $|x|$; it can be thought of as two $|x|$ by $|x|$ matrices stacked vertically:
    * The top matrix $T$ is such that $Tx$ produces a vector $v$ where $v_j$ is the net change in the normal velocity of the contact represented by $x_j$ ($v_j > 0 \implies$ its points were pushed towards each other and $v_j < 0 \implies$ its points were pushed away from each other).
    * The bottom matrix is the identity matrix multiplied by $-1$.
* $h$ is a vector of cardinality $2|x|$; it can be thought of as two vectors of cardinality $|x|$ stacked vertically:
    * The top vector $t$ is such that $t_j$ is the "normal velocity cushion" of the contact represented by $x_j$ before applying any impulses ($t_j > 0 \implies$ its points can be pushed toward each other by up to $t_j$ and $t_j < 0 \implies$ its points must be pushed away from each other by at least $t_j$). It is computed as $t_j = \textrm{velocityDifference}(x_j)\cdot\textrm{unitNormalVector}(x_j) + \textrm{distance}(x_j)*\textrm{timestep}$.
    * The bottom vector is the zero vector.
* Therefore, the effects of the $Gx \leq h$ constraint are that:
    * The net change in normal velocity of each contact will be such that the no contacts end in a colliding state.
    * All impulses applied to contacts will be nonattractive.
* $P$ is an $|x|$ by $|x|$ matrix equal to $-2T$.
* $q$ is a vector of cardinality $|x|$ equal to $t$.
* The objective can therefore rewritten as $x \cdot (-Tx + t)$.
* This is equivalent to $x \cdot \textrm{(normal velocity cushion of the contact represented by } x_j \textrm{ after applying all impulses, guranteed to be nonnegative by the constraint)}$.
* Minimizing this minimizes overcorrection because the solver will minimize the cushion of all contacts that have had corrections applied to them (contacts that haven't had corrections applied to them have $x_j = 0$, so it won't need to minimize those cushions).

We implemented all three of these methods in Python (using the [cvxopt.qp](https://cvxopt.org/userguide/coneprog.html#quadratic-programming) solver for `LCP`). To to stress-test their support for fast-moving objects and stable stacks, we had them simulate the high-speed stacking of 18 two-dimensional discs. Henceforth, these discs will be referred to as "gecs" in tribute to the experimental pop band [100 gecs](https://soundcloud.com/100gecs/i-need-help-immediately).

### Python Code

In [ ]:
#TODO python code

### GAMS Code (to double-check our Python optimization results)

In [ ]:
#TODO python code

## Results

### greedy

In [15]:
HTML('<iframe width="700" height="520" src="https://gfycat.com/ifr/unlawfulbaggydalmatian?&hd=1&autoplay=0">')

### greedyWithCorrections(3)

In [16]:
HTML('<iframe width="700" height="520" src="https://gfycat.com/ifr/ignorantgrandioseargusfish?&hd=1&autoplay=0">')

### LCP

In [18]:
HTML('<iframe width="700" height="520" src="https://gfycat.com/ifr/athleticwanguanaco?&hd=1&autoplay=0">')